In [1]:
import pandas as pd
import networkx as nx
from pathlib import Path
from networkx.drawing.nx_agraph import graphviz_layout

prefix = 'data/umimematikucz-system_'

kc = pd.read_csv(Path(prefix + "kc.csv", ), sep=';', header=0, index_col=0)
follow = pd.read_csv(Path(prefix + "kc_follow.csv", ), sep=';', header=0, index_col=0)
ps = pd.read_csv(Path(prefix + "ps.csv", ), sep=';', header=0, index_col=0)

G = nx.Graph()

G.add_nodes_from([(idx, {'name':name}) for idx, name in zip(kc.index, kc['name'])])
G.add_node(0, name='root')
G.add_edges_from([(idx, parent) for idx, parent in zip(kc.index, kc['parent'])])
G.add_edges_from([(first, second) for first, second in follow[['kc1', 'kc2']].to_numpy()])
pos = nx.spectral_layout(G, scale=400, center=(200, 200))
# pos = graphviz_layout(G, prog='dot')

In [2]:
def encode_node(idx, name, position):
    return {'data': {'id': str(idx), 'label': name}, 'position': {'x': int(position[0]), 'y': int(position[1])}}

def encode_edge(source, target):
    return {'data': {'source': str(source), 'target': str(target)}}

elements = []

elements += [encode_node(idx, params['name'], pos[idx]) for idx, params in G.nodes(data=True)]
elements += [encode_edge(s, t) for s, t in G.edges()]


In [3]:
import json
import dash_cytoscape as cyto
import dash_bootstrap_components as dbc
import src.custom_components as cc
from dash import dcc, html, Input, Output, callback_context
from jupyter_dash import JupyterDash
from dash.exceptions import PreventUpdate

app = JupyterDash(__name__)

styles = json.load(open('styles.json', 'r'))

cyto.load_extra_layouts()

app.layout = cc.place_in_container([
    dbc.Row([
        html.Div(cyto.Cytoscape(
            id='cytoscape',
            elements=elements,
            style=styles['cytoscape'],
            layout={'name': 'klay'}, # klay
            responsive=True,
            autoRefreshLayout=False
        )),
    ]),
    dbc.Row([
        dcc.Dropdown(
            id='graph_layout',
            value='klay',
            clearable=False,
            options=[
                {'label': name.capitalize(), 'value': name}
                for name in ['klay', 'random', 'cose', 'concentric', 'breadthfirst']
            ]
        )
    ]),
    html.Div(id = 'last_clicked'), # , style = {'display': 'none'}
])

In [4]:
import json
"""
@app.callback(
        Output('last_clicked', 'children'),
        # Input('last_clicked', 'children'),
        Input('cytoscape', 'tapNodeData'),
)
def update_last_clicked(clicked):
    return json.dumps(clicked)
"""

@app.callback(
        Output('last_clicked', 'children'),
        Input('cytoscape', 'tapNodeData'),
        Input('cytoscape', 'elements'),
        Input('cytoscape', 'selectedNodeData')
)
def update_last_clicked(clicked, elements, selected):
    if callback_context.triggered and callback_context.triggered[0]['prop_id'].split('.')[1] != 'tapNodeData':
        raise PreventUpdate
    return json.dumps(elements[-1])


def filter_for_edge(edge):
    return lambda e: 'source' not in e['data'] or e['data']['source'] != edge['data']['source'] or e['data']['target'] != edge['data']['target']


@app.callback(
        Output('cytoscape', 'elements'),
        Input('cytoscape', 'tapNodeData'),
        Input('cytoscape', 'elements'),
        Input('cytoscape', 'selectedNodeData')
)
def update_graph(clicked, elements, selected):
    if callback_context.triggered and callback_context.triggered[0]['prop_id'].split('.')[1] != 'tapNodeData' or not selected:
        raise PreventUpdate
    
    edge = encode_edge(int(selected[-1]['id']), clicked['id'])
    old_length = len(elements)
    elements = list(filter(filter_for_edge(edge), elements))
    if len(elements) == old_length:
        elements.append(edge)
        
    return elements
    # TODO adding and removing edges
    # TODO layout
    # TODO nodes?
    # TODO dfs order after doubleclick

@app.callback(
    Output('cytoscape', 'layout'),
    Input('graph_layout', 'value')
)
def update_layout(layout):
    return {
        'name': layout,
        'animate': True
    }

In [5]:
if __name__ == "__main__":
    app.run_server(debug=True)

Dash app running on http://127.0.0.1:8050/
